# Simulating a cricket game

Seperately from the previous modeling, where I tried to predict the outcome of a game based on the stats of the players, I want to attempt to model a cricket game itself, ball by ball.

I will create three models:
1. A model that predicts the probability of a wicket occuring in a ball.
2. A model predicting the number of extras resulting from a ball.
3. A model predicting the number of runs scored by a ball.

These will then be combined into a cricket simulator with the aim of predicting the outcome of a game through modelling the sequence of events within it.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ast
from joblib import dump, load
import traceback
import sys

from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_absolute_percentage_error, f1_score
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import OneHotEncoder

# 1. Predicting the probability of a wicket occuring in a ball

In [2]:
data = pd.read_csv('../data/saved_data/all_matches.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 857199 entries, 0 to 857198
Data columns (total 29 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   game_id               857199 non-null  object 
 1   date                  857199 non-null  object 
 2   venue                 857199 non-null  object 
 3   location              816895 non-null  object 
 4   gender                857199 non-null  object 
 5   match_type            857199 non-null  object 
 6   innings               857199 non-null  int64  
 7   batting_team          857199 non-null  object 
 8   bowling_team          857199 non-null  object 
 9   batting_team_players  857199 non-null  object 
 10  bowling_team_players  857199 non-null  object 
 11  over                  857199 non-null  int64  
 12  ball_in_over          857199 non-null  int64  
 13  batter                857199 non-null  object 
 14  bowler                857199 non-null  object 
 15  

In [3]:
data['extras_details'] = data['extras_details'].apply(lambda x: ast.literal_eval(x) if not pd.isna(x) else {})
data['freehit'] = data['extras_details'].apply(lambda x: 'noballs' in list(x.keys()))
data['freehit'].value_counts()

freehit
False    852523
True       4676
Name: count, dtype: int64

In [4]:
# Before I do anything, I am going to seperate my data into training and testing sets.
# The training set will be used to train all the models/make distributions for the simulator
# The test set will be used to test the overall model at the end.


unique_games = data['game_id'].unique()
train_games, test_games = train_test_split(unique_games, test_size=0.2, random_state=42)

train_data = data[data['game_id'].isin(train_games)]
test_data = data[data['game_id'].isin(test_games)]

test_data.to_csv('../data/saved_data/simulation_test_data.csv', index=False)
data = train_data

In [5]:
batter_stats = pd.read_csv('../data/saved_data/batter_stats.csv')
bowler_stats = pd.read_csv('../data/saved_data/bowler_stats.csv')

In [6]:
batter_stats.head()

,batter,total_runs_batter_mean,total_high_scoring_hit_mean,total_total_mean,total_is_wicket_mean,powerplay_runs_batter_mean,powerplay_high_scoring_hit_mean,powerplay_total_mean,powerplay_is_wicket_mean,non_powerplay_runs_batter_mean,non_powerplay_high_scoring_hit_mean,non_powerplay_total_mean,non_powerplay_is_wicket_mean
0,00015688,0.294118,0.029412,0.485294,0.044118,0.375,0.05,0.525,0.025,0.178571,0.000000,0.428571,0.071429
1,00029c30,0.454545,0.000000,0.818182,0.000000,NaN,NaN,NaN,NaN,0.454545,0.000000,0.818182,0.000000
2,0030a57d,0.533333,0.000000,0.733333,0.000000,NaN,NaN,NaN,NaN,0.533333,0.000000,0.733333,0.000000
3,00321fff,0.612245,0.040816,0.673469,0.081633,NaN,NaN,NaN,NaN,0.612245,0.040816,0.673469,0.081633
4,00467a76,0.974684,0.101266,1.088608,0.101266,NaN,NaN,NaN,NaN,0.974684,0.101266,1.088608,0.101266


In [7]:
bowler_stats.head()

,bowler,total_batter_runs_conceded_mean,total_runs_from_relevant_extras_mean,total_total_runs_conceded_mean,total_taken_from_relevant_wickets_mean,powerplay_batter_runs_conceded_mean,powerplay_runs_from_relevant_extras_mean,powerplay_total_runs_conceded_mean,powerplay_taken_from_relevant_wickets_mean,non_powerplay_batter_runs_conceded_mean,non_powerplay_runs_from_relevant_extras_mean,non_powerplay_total_runs_conceded_mean,non_powerplay_taken_from_relevant_wickets_mean
0,00029c30,1.090909,0.060606,1.151515,0.045455,NaN,NaN,NaN,NaN,1.090909,0.060606,1.151515,0.045455
1,00321fff,1.067210,0.036660,1.107943,0.052953,NaN,NaN,NaN,NaN,1.067210,0.036660,1.107943,0.052953
2,00467a76,1.236742,0.068182,1.348485,0.066288,1.331034,0.048276,1.427586,0.055172,1.201044,0.075718,1.318538,0.070496
3,005f0561,0.864734,0.021739,0.898551,0.057971,0.757396,0.017751,0.775148,0.041420,0.938776,0.024490,0.983673,0.069388
4,007113d7,1.008584,0.025751,1.068670,0.081545,NaN,NaN,NaN,NaN,1.008584,0.025751,1.068670,0.081545


In [8]:
powerplay_balls = data[data['powerplay'] == True][['batter', 'bowler', 'is_wicket', 'freehit']]
powerplay_batter_stats = batter_stats[['batter'] + [col for col in batter_stats.columns if 'powerplay' in col[:9]]]
powerplay_bowler_stats = bowler_stats[['bowler'] + [col for col in bowler_stats.columns if 'powerplay' in col[:9]]]

non_powerplay_balls = data[data['powerplay'] == False][['batter', 'bowler', 'is_wicket', 'freehit']]
non_powerplay_batter_stats = batter_stats[['batter'] + [col for col in batter_stats.columns if 'non_powerplay' in col[:13]]]
non_powerplay_bowler_stats = bowler_stats[['bowler'] + [col for col in bowler_stats.columns if 'non_powerplay' in col[:13]]]

Due to time constraints, their performance and ease of use, I will be using a random forest classifierto predict the probability of a wicket occuring in a ball. It's ability to easily return class probabilities rather than just the classes makes it useful here too.

In [9]:
powerplay_balls_stats = powerplay_balls.merge(powerplay_batter_stats, on='batter', how='left').drop(columns=['batter'])
powerplay_balls_stats = powerplay_balls_stats.merge(powerplay_bowler_stats, on='bowler', how='left').drop(columns=['bowler'])

non_powerplay_balls_stats = non_powerplay_balls.merge(non_powerplay_batter_stats, on='batter', how='left').drop(columns=['batter'])
non_powerplay_balls_stats = non_powerplay_balls_stats.merge(non_powerplay_bowler_stats, on='bowler', how='left').drop(columns=['bowler'])
powerplay_balls_stats.head()

,is_wicket,freehit,powerplay_runs_batter_mean,powerplay_high_scoring_hit_mean,powerplay_total_mean,powerplay_is_wicket_mean,powerplay_batter_runs_conceded_mean,powerplay_runs_from_relevant_extras_mean,powerplay_total_runs_conceded_mean,powerplay_taken_from_relevant_wickets_mean
0,False,False,1.329810,0.228330,1.439746,0.033827,0.65942,0.144928,0.855072,0.050725
1,False,False,1.329810,0.228330,1.439746,0.033827,0.65942,0.144928,0.855072,0.050725
2,False,False,1.329810,0.228330,1.439746,0.033827,0.65942,0.144928,0.855072,0.050725
3,False,False,1.268603,0.215971,1.333938,0.050817,0.65942,0.144928,0.855072,0.050725
4,False,False,1.268603,0.215971,1.333938,0.050817,0.65942,0.144928,0.855072,0.050725


In [10]:
X_powerplay = powerplay_balls_stats.drop(columns=['is_wicket'])
y_powerplay = powerplay_balls_stats['is_wicket']

X_train, X_test, y_train, y_test = train_test_split(X_powerplay, y_powerplay, test_size=0.2, random_state=42)

In [11]:
from sklearn.ensemble import RandomForestClassifier

def do_random_forest_classifier(X_train, y_train, cv=5, random_state=42, **kwargs):
    """
    Train Random Forest Classifier and get probability predictions
    """
    model = RandomForestClassifier(
        random_state=random_state,
        **kwargs
    )
    cv_results = cross_validate(
        model,
        X_train,
        y_train,
        cv=cv,
        scoring={
            'auc': 'roc_auc',
            'precision': 'precision', 
            'recall': 'recall',
            'f1': 'f1'
        },
        return_train_score=True,
        verbose=1
    )
    
    model.fit(X_train, y_train)
    
    print("Model Performance:")
    print(f"Train AUC: {cv_results['train_auc'].mean():.3f} ± {cv_results['train_auc'].std():.3f}")
    print(f"CV AUC: {cv_results['test_auc'].mean():.3f} ± {cv_results['test_auc'].std():.3f}")

    print(f"Precision: {cv_results['test_precision'].mean():.3f} ± {cv_results['test_precision'].std():.3f}")
    print(f"Recall: {cv_results['test_recall'].mean():.3f} ± {cv_results['test_recall'].std():.3f}")
    print(f"F1 Score: {cv_results['test_f1'].mean():.3f} ± {cv_results['test_f1'].std():.3f}")
    
    return model

powerplay_wicket_model = do_random_forest_classifier(X_train, y_train)

Model Performance:
Train AUC: 0.963 ± 0.000
CV AUC: 0.598 ± 0.002
Precision: 0.043 ± 0.008
Recall: 0.010 ± 0.002
F1 Score: 0.016 ± 0.003


In [12]:
dump(powerplay_wicket_model, '../models/simulation_models/powerplay_wicket_model.joblib')

['../models/simulation_models/powerplay_wicket_model.joblib']

In [13]:
powerplay_results = powerplay_wicket_model.predict_proba(X_test)
powerplay_results


array([[1.        , 0.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       ...,
       [0.87471492, 0.12528508],
       [0.93141667, 0.06858333],
       [1.        , 0.        ]])

In [14]:
print(f"Proportion of balls where wicket chance is 0: {len(powerplay_results[powerplay_results[:, 1] == 0]) / len(powerplay_results):.3f}")
print(f"Max wicket chance: {powerplay_results[:, 1].max():.3f}")
print(f"Mean wicket chance: {powerplay_results[:, 1].mean():.3f}")

Proportion of balls where wicket chance is 0: 0.574
Max wicket chance: 0.941
Mean wicket chance: 0.041


At a face value these results look awful, and were we trying to use this to classify whether or not a wicket occurs they would be.

However, we are trying to predict the probability of a wicket occuring in a ball. Binary classification models return the class with the highest proability - which in our case will pretty much always be no wicket...

Therefore, I do not think the low auc/f1 values are a problem.

I am however more concerned about the high number of balls with a wicket chance of 0. Since technically, there is always a small chance of a wicket occuring, I am going to add a small 'fudge factor' to the predictions to ensure that this always exists.

In [15]:
print(f"Mean actual wicket chance: {powerplay_balls_stats['is_wicket'].mean():.3f}")
print(f"Mean predicted wicket chance: {powerplay_results[:, 1].mean():.3f}")

print(f"STD of actual wicket chance: {powerplay_balls_stats['is_wicket'].std():.3f}")
print(f"STD of predicted wicket chance: {powerplay_results[:, 1].std():.3f}")

Mean actual wicket chance: 0.044
Mean predicted wicket chance: 0.041
STD of actual wicket chance: 0.205
STD of predicted wicket chance: 0.097


The mean wicket chance outputted by the model is 0.04, very similar to the actual wicket chance of 0.039, suggesting that the model is actually fairly representative of the actual wicket chance.

Therefore I probably want the fudge factor to be small but non-zero, keeping the mean predicted wicket chance close to the actual wicket chance.


In [16]:
def predict_wicket_probability(model, X, fudge_factor=0.01):
    wicket_prob = model.predict_proba(X)[:, 1]
    wicket_prop = wicket_prob * (1 - fudge_factor) + fudge_factor
    return wicket_prop

powerplay_results = predict_wicket_probability(powerplay_wicket_model, X_test, 0.01)
print(f"New mean predicted wicket chance: {powerplay_results.mean():.3f}")

New mean predicted wicket chance: 0.051


Adding a fudge factor of 1% seems to be alright. While it is arbitrarily chosen, it does keep the mean chance similar to the actual chance while ensuring that there is somewhat of a chance of a wicket occuring at all times.

# Non powerplay wickets

Again, I am going to repeat the exact same process for the non powerplay wickets.

In [17]:
X_non_powerplay = non_powerplay_balls_stats.drop(columns=['is_wicket'])
y_non_powerplay = non_powerplay_balls_stats['is_wicket']

X_train, X_test, y_train, y_test = train_test_split(X_non_powerplay, y_non_powerplay, test_size=0.2, random_state=42)

In [18]:
non_powerplay_wicket_model = do_random_forest_classifier(X_train, y_train)

Model Performance:
Train AUC: 0.955 ± 0.000
CV AUC: 0.579 ± 0.004
Precision: 0.056 ± 0.003
Recall: 0.014 ± 0.001
F1 Score: 0.022 ± 0.001


In [19]:
dump(non_powerplay_wicket_model, '../models/simulation_models/non_powerplay_wicket_model.joblib')

['../models/simulation_models/non_powerplay_wicket_model.joblib']

In [20]:
non_powerplay_results = predict_wicket_probability(non_powerplay_wicket_model, X_test, 0)
print(f"Proportion of balls where wicket chance is 0: {len(non_powerplay_results[non_powerplay_results == 0]) / len(non_powerplay_results):.3f}")
print(f"Max wicket chance: {non_powerplay_results.max():.3f}")
print(f"Mean wicket chance: {non_powerplay_results.mean():.3f}")

Proportion of balls where wicket chance is 0: 0.489
Max wicket chance: 0.990
Mean wicket chance: 0.056


In [21]:
print(f"Mean actual wicket chance: {non_powerplay_balls_stats['is_wicket'].mean():.3f}")
print(f"Mean predicted wicket chance: {non_powerplay_results.mean():.3f}")

print(f"STD of actual wicket chance: {non_powerplay_balls_stats['is_wicket'].std():.3f}")
print(f"STD of predicted wicket chance: {non_powerplay_results.std():.3f}")


Mean actual wicket chance: 0.060
Mean predicted wicket chance: 0.056
STD of actual wicket chance: 0.238
STD of predicted wicket chance: 0.113


Again we see that the model is fairly representative of the actual wicket chance, good signs all around.

I think I will stick with the same fudge factor of 1% for the non powerplay wickets to keep things consistent.

In [22]:
non_powerplay_results = predict_wicket_probability(non_powerplay_wicket_model, X_test, 0.01)
print(f"New mean predicted wicket chance: {non_powerplay_results.mean():.3f}")

New mean predicted wicket chance: 0.066


Again, this remains within ±1% of the actual wicket chance, I'm happy with this for now.

---
# 2. Predicting the number of extras resulting from a ball

I think I am going to break this down into three models:
1. A model predicting the probability of an extra occuring in a ball.
2. A model predicting the type of extra | an extra has occured.
3. A model predicting the number of extras occuring in a ball | The type of extra

This should help solve the problem of the high number of extra balls where the number of extras is 0, which would likely skew the predictions if I were to use just a single model.

Again, I am going to use a random forest classifier for this task.

## 2.a) Predicting the probability of an extra occuring in a ball

## Powerplay

In [23]:
powerplay_extra_balls = data[data['powerplay'] == True][['batter', 'bowler', 'extras']]
powerplay_extra_batter_stats = batter_stats[['batter'] + [col for col in batter_stats.columns if 'powerplay' in col[:9]]]
powerplay_extra_bowler_stats = bowler_stats[['bowler'] + [col for col in bowler_stats.columns if 'powerplay' in col[:9]]]

non_powerplay_extra_balls = data[data['powerplay'] == False][['batter', 'bowler', 'extras']]
non_powerplay_extra_batter_stats = batter_stats[['batter'] + [col for col in batter_stats.columns if 'non_powerplay' in col[:13]]]
non_powerplay_extra_bowler_stats = bowler_stats[['bowler'] + [col for col in bowler_stats.columns if 'non_powerplay' in col[:13]]]

In [24]:
powerplay_extra_balls['extras'].mean()

np.float64(0.08995126113550889)

In [25]:
powerplay_extra_balls['is_extras'] = powerplay_extra_balls['extras'] > 0
powerplay_extra_balls.info()

<class 'pandas.core.frame.DataFrame'>
Index: 215639 entries, 248 to 857110
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   batter     215639 non-null  object
 1   bowler     215639 non-null  object
 2   extras     215639 non-null  int64 
 3   is_extras  215639 non-null  bool  
dtypes: bool(1), int64(1), object(2)
memory usage: 6.8+ MB


In [26]:
powerplay_extra_balls = powerplay_extra_balls.merge(powerplay_extra_batter_stats, on='batter', how='left').drop(columns=['batter'])
powerplay_extra_balls = powerplay_extra_balls.merge(powerplay_extra_bowler_stats, on='bowler', how='left').drop(columns=['bowler'])

In [27]:
X_powerplay = powerplay_extra_balls.drop(columns=['extras', 'is_extras'])
y_powerplay = powerplay_extra_balls['is_extras']

X_train, X_test, y_train, y_test = train_test_split(X_powerplay, y_powerplay, test_size=0.2, random_state=42)

In [28]:
extras_model = do_random_forest_classifier(X_train, y_train)

Model Performance:
Train AUC: 0.924 ± 0.001
CV AUC: 0.585 ± 0.008
Precision: 0.140 ± 0.013
Recall: 0.034 ± 0.005
F1 Score: 0.055 ± 0.007


In [29]:
dump(extras_model, '../models/simulation_models/powerplay_extras_binary_model.joblib')

['../models/simulation_models/powerplay_extras_binary_model.joblib']

In [30]:
powerplay_results = predict_wicket_probability(extras_model, X_test, 0)
print(f"Proportion of balls where extra chance is 0: {len(powerplay_results[powerplay_results == 0]) / len(powerplay_results):.3f}")
print(f"Max extra chance: {powerplay_results.max():.3f}")
print(f"Mean extra chance: {powerplay_results.mean():.3f}")

Proportion of balls where extra chance is 0: 0.424
Max extra chance: 0.993
Mean extra chance: 0.072


In [31]:
print(f"Mean actual extra chance: {powerplay_extra_balls['extras'].mean():.3f}")
print(f"Mean predicted extra chance: {powerplay_results.mean():.3f}")

print(f"STD of actual extra chance: {powerplay_extra_balls['extras'].std():.3f}")
print(f"STD of predicted extra chance: {powerplay_results.std():.3f}")

Mean actual extra chance: 0.090
Mean predicted extra chance: 0.072
STD of actual extra chance: 0.392
STD of predicted extra chance: 0.123


Interesting to see that the predictions are less accurate here than for the wickets. Its still within ±2% of the actual mean chance, but not as close as I would like.

That being said, again adding a fudge factor of 1% seems like it would be alright, and in fact would bring the mean predicted extra chance closer to the actual extra chance.

In [32]:
powerplay_results = predict_wicket_probability(extras_model, X_test, 0.01)
print(f"New mean predicted extra chance: {powerplay_results.mean():.3f}")

New mean predicted extra chance: 0.081


## Non powerplay

In [33]:
non_powerplay_extra_balls['is_extras'] = non_powerplay_extra_balls['extras'] > 0
non_powerplay_extra_balls = non_powerplay_extra_balls.merge(non_powerplay_extra_batter_stats, on='batter', how='left').drop(columns=['batter'])
non_powerplay_extra_balls = non_powerplay_extra_balls.merge(non_powerplay_extra_bowler_stats, on='bowler', how='left').drop(columns=['bowler'])
X_non_powerplay = non_powerplay_extra_balls.drop(columns=['extras', 'is_extras'])
y_non_powerplay = non_powerplay_extra_balls['is_extras']

X_train, X_test, y_train, y_test = train_test_split(X_non_powerplay, y_non_powerplay, test_size=0.2, random_state=42)
non_powerplay_extras_model = do_random_forest_classifier(X_train, y_train)
non_powerplay_results = predict_wicket_probability(non_powerplay_extras_model, X_test, 0)
print(f"Proportion of balls where extra chance is 0: {len(non_powerplay_results[non_powerplay_results == 0]) / len(non_powerplay_results):.3f}")
print(f"Max extra chance: {non_powerplay_results.max():.3f}")
print(f"Mean extra chance: {non_powerplay_results.mean():.3f}")
print(f"\nMean actual extra chance: {non_powerplay_extra_balls['extras'].mean():.3f}")
print(f"Mean predicted extra chance: {non_powerplay_results.mean():.3f}")

print(f"STD of actual extra chance: {non_powerplay_extra_balls['extras'].std():.3f}")
print(f"STD of predicted extra chance: {non_powerplay_results.std():.3f}")


Model Performance:
Train AUC: 0.935 ± 0.000
CV AUC: 0.580 ± 0.003
Precision: 0.123 ± 0.007
Recall: 0.029 ± 0.002
F1 Score: 0.047 ± 0.003
Proportion of balls where extra chance is 0: 0.456
Max extra chance: 0.995
Mean extra chance: 0.063

Mean actual extra chance: 0.076
Mean predicted extra chance: 0.063
STD of actual extra chance: 0.349
STD of predicted extra chance: 0.117


In [34]:
dump(non_powerplay_extras_model, '../models/simulation_models/non_powerplay_extras_binary_model.joblib')

['../models/simulation_models/non_powerplay_extras_binary_model.joblib']

Again, a fudge factor of 1% seems appropriate here.

In [35]:
non_powerplay_results = predict_wicket_probability(non_powerplay_extras_model, X_test, 0.01)
print(f"New mean predicted extra chance: {non_powerplay_results.mean():.3f}")

New mean predicted extra chance: 0.072


# 2.b) Predicting the type of extra | an extra has occured

My hope here is that using the batter and bowler stats we can predict the type of extra that occurs.

Depending on the success of this, I may instead just try to predict the number of runs|extra occured.

In [36]:
balls_with_extras = data[data['extras'] > 0][['batter', 'bowler', 'extras_details', 'extras', 'powerplay']]

In [37]:
balls_with_extras['extras_details'].value_counts()

extras_details
{'wides': 1}                    25938
{'legbyes': 1}                   7542
{'noballs': 1}                   3567
{'byes': 1}                      1936
{'wides': 2}                     1705
{'wides': 5}                      809
{'legbyes': 2}                    611
{'byes': 4}                       578
{'legbyes': 4}                    496
{'wides': 3}                      429
{'byes': 2}                       336
{'byes': 1, 'noballs': 1}          91
{'wides': 4}                       47
{'legbyes': 3}                     46
{'byes': 4, 'noballs': 1}          35
{'byes': 3}                        28
{'legbyes': 1, 'noballs': 1}       20
{'byes': 2, 'noballs': 1}          16
{'legbyes': 5}                     11
{'penalty': 5}                     10
{'noballs': 5}                      9
{'noballs': 2}                      9
{'legbyes': 4, 'noballs': 1}        4
{'legbyes': 2, 'noballs': 1}        2
{'byes': 5}                         2
{'legbyes': 1, 'penalty': 5}       

In [38]:
balls_with_extras['extras_details'].apply(lambda x: list(x.keys())).value_counts()

extras_details
[wides]               28928
[legbyes]              8706
[noballs]              3585
[byes]                 2880
[byes, noballs]         132
[legbyes, noballs]       25
[penalty]                10
[noballs, byes]          10
[legbyes, penalty]        1
[noballs, legbyes]        1
[noballs, penalty]        1
Name: count, dtype: int64

In [39]:
balls_with_extras.shape

(44279, 5)

In [40]:
noballs_balls = balls_with_extras[balls_with_extras['extras_details'].apply(lambda x: list(x.keys()) == ['noballs'])]
noballs_balls['extras_details'].value_counts()

extras_details
{'noballs': 1}    3567
{'noballs': 5}       9
{'noballs': 2}       9
Name: count, dtype: int64

We have a couple if issues here:
1. If we are trying to predict the number of extra runs | extra occurs, the results are massively skewed towards 1.
2. If we are trying to predict the number of extra runs | extra type the results will still heavily be skewed towards 1 for the extras from which there can be a variable number of runs.

I think this is a lose-lose situation, but at least predicting the probability of an extra type should help in the case of extras where there is a fixed number of runs.

In [41]:
balls_with_extras['extras_type'] = balls_with_extras['extras_details'].apply(lambda x: list(x.keys()))
exploded_extras = balls_with_extras.explode('extras_type')
exploded_extras['extras_type'].value_counts()

extras_type
wides      28928
legbyes     8733
noballs     3754
byes        3022
penalty       12
Name: count, dtype: int64

We have 5 types of extras. Since we can have multiple extras in a ball, I am going to use 5 different models to predict the probability of each type occuring.

For the sake of length and readability, I am going to limit my explanations/analysis.

In [42]:
# Unfortuntely due to the way the data is structured I can't use one_hot_encoding with the exploded rows
# without duplicating rows, so I will just create the columns manually.
balls_with_extras['wides'] = balls_with_extras['extras_details'].apply(lambda x: 'wides' in list(x.keys()))
balls_with_extras['byes'] = balls_with_extras['extras_details'].apply(lambda x: 'byes' in list(x.keys()))
balls_with_extras['legbyes'] = balls_with_extras['extras_details'].apply(lambda x: 'legbyes' in list(x.keys()))
balls_with_extras['noballs'] = balls_with_extras['extras_details'].apply(lambda x: 'noballs' in list(x.keys()))
balls_with_extras['penalty'] = balls_with_extras['extras_details'].apply(lambda x: 'penalty' in list(x.keys()))
extras_dropped = balls_with_extras.drop(columns=['extras_details', 'extras_type', 'extras'])

powerplay_extra_balls = extras_dropped[extras_dropped['powerplay']].drop(columns=['powerplay'])
powerplay_extra_balls = powerplay_extra_balls.merge(powerplay_extra_batter_stats, on='batter', how='left').drop(columns=['batter'])
powerplay_extra_balls = powerplay_extra_balls.merge(powerplay_extra_bowler_stats, on='bowler', how='left').drop(columns=['bowler'])

non_powerplay_extra_balls = extras_dropped[extras_dropped['powerplay'] == False].drop(columns=['powerplay'])
non_powerplay_extra_balls = non_powerplay_extra_balls.merge(non_powerplay_extra_batter_stats, on='batter', how='left').drop(columns=['batter'])
non_powerplay_extra_balls = non_powerplay_extra_balls.merge(non_powerplay_extra_bowler_stats, on='bowler', how='left').drop(columns=['bowler'])

In [43]:
def get_empirical_distribution(data):
    """
    Create probability distribution directly from data
    
    Args:
        data: Series of values
    
    Returns:
        values: possible values
        probabilities: probability of each value
    """
    # Get value counts and convert to probabilities
    value_counts = data.value_counts()
    total = len(data)
    probabilities = value_counts / total
    
    # Sort by value for clarity
    probabilities = probabilities.sort_index()
    
    return probabilities

In [44]:
# Wides:
# For the sake of time I am going to limit the max depth of the trees to 30, otherwise I will be sitting here for hours...
X_powerplay_wides = powerplay_extra_balls.drop(columns=['wides', 'byes', 'legbyes', 'noballs', 'penalty'])
y_powerplay_wides = powerplay_extra_balls['wides']
X_train, X_test, y_train, y_test = train_test_split(X_powerplay_wides, y_powerplay_wides, test_size=0.2, random_state=42)
print(f"Powerplay wides model:")
powerplay_wides_model = do_random_forest_classifier(X_train, y_train, max_depth=30)
test_wides = predict_wicket_probability(powerplay_wides_model, X_test, 0)

# Byes:
X_powerplay_byes = powerplay_extra_balls.drop(columns=['wides', 'byes', 'legbyes', 'noballs', 'penalty'])
y_powerplay_byes = powerplay_extra_balls['byes']
X_train, X_test, y_train, y_test = train_test_split(X_powerplay_byes, y_powerplay_byes, test_size=0.2, random_state=42)
print(f"Powerplay byes model:")
powerplay_byes_model = do_random_forest_classifier(X_train, y_train, max_depth=30)
test_byes = predict_wicket_probability(powerplay_byes_model, X_test, 0)
# Legbyes:
X_powerplay_legbyes = powerplay_extra_balls.drop(columns=['wides', 'byes', 'legbyes', 'noballs', 'penalty'])
y_powerplay_legbyes = powerplay_extra_balls['legbyes']
X_train, X_test, y_train, y_test = train_test_split(X_powerplay_legbyes, y_powerplay_legbyes, test_size=0.2, random_state=42)
print(f"Powerplay legbyes model:")
powerplay_legbyes_model = do_random_forest_classifier(X_train, y_train, max_depth=30)
test_legbyes = predict_wicket_probability(powerplay_legbyes_model, X_test, 0)

# Noballs:
X_powerplay_noballs = powerplay_extra_balls.drop(columns=['wides', 'byes', 'legbyes', 'noballs', 'penalty'])
y_powerplay_noballs = powerplay_extra_balls['noballs']
X_train, X_test, y_train, y_test = train_test_split(X_powerplay_noballs, y_powerplay_noballs, test_size=0.2, random_state=42)
print(f"Powerplay noballs model:")
powerplay_noballs_model = do_random_forest_classifier(X_train, y_train, max_depth=30)
test_noballs = predict_wicket_probability(powerplay_noballs_model, X_test, 0)
# Penalties:
# There are no penalites in the powerplay data, and only 14 in the non powerplay data.
# I am hence going to just model this as a random chance, especially considering that the batter/bolwer skill
# in theory should have no impact on the probability of a penalty occuring.
penalty_distribution = get_empirical_distribution(non_powerplay_extra_balls['penalty'])
penalty_distribution.to_csv('../models/simulation_dists/penalty_distribution.csv', header=False)
print(f"Mean penalty chance: {penalty_distribution.mean():.3f}")

Powerplay wides model:
Model Performance:
Train AUC: 0.988 ± 0.000
CV AUC: 0.613 ± 0.008
Precision: 0.723 ± 0.003
Recall: 0.842 ± 0.009
F1 Score: 0.778 ± 0.004
Powerplay byes model:
Model Performance:
Train AUC: 0.998 ± 0.000
CV AUC: 0.598 ± 0.015
Precision: 0.097 ± 0.054
Recall: 0.026 ± 0.014
F1 Score: 0.041 ± 0.023
Powerplay legbyes model:
Model Performance:
Train AUC: 0.992 ± 0.000
CV AUC: 0.650 ± 0.009
Precision: 0.343 ± 0.018
Recall: 0.158 ± 0.011
F1 Score: 0.216 ± 0.012
Powerplay noballs model:
Model Performance:
Train AUC: 0.995 ± 0.000
CV AUC: 0.621 ± 0.023
Precision: 0.282 ± 0.025
Recall: 0.097 ± 0.007
F1 Score: 0.144 ± 0.009
Mean penalty chance: 0.500


In [45]:
dump(powerplay_wides_model, '../models/simulation_models/powerplay_wides_model.joblib')
dump(powerplay_byes_model, '../models/simulation_models/powerplay_byes_model.joblib')
dump(powerplay_legbyes_model, '../models/simulation_models/powerplay_legbyes_model.joblib')
dump(powerplay_noballs_model, '../models/simulation_models/powerplay_noballs_model.joblib')

['../models/simulation_models/powerplay_noballs_model.joblib']

In [46]:
print(f"Mean actual wides chance: {powerplay_extra_balls['wides'].mean():.3f}")
print(f"Mean predicted wides chance: {test_wides.mean():.3f}")
print(f"Proportion of balls where wides chance is 0: {len(test_wides[test_wides == 0]) / len(test_wides):.3f}\n")

print(f"Mean actual byes chance: {powerplay_extra_balls['byes'].mean():.3f}")
print(f"Mean predicted byes chance: {test_byes.mean():.3f}")
print(f"Proportion of balls where byes chance is 0: {len(test_byes[test_byes == 0]) / len(test_byes):.3f}\n")
print(f"Mean actual legbyes chance: {powerplay_extra_balls['legbyes'].mean():.3f}")
print(f"Mean predicted legbyes chance: {test_legbyes.mean():.3f}")
print(f"Proportion of balls where legbyes chance is 0: {len(test_legbyes[test_legbyes == 0]) / len(test_legbyes):.3f}\n")

print(f"Mean actual noballs chance: {powerplay_extra_balls['noballs'].mean():.3f}")
print(f"Mean predicted noballs chance: {test_noballs.mean():.3f}")
print(f"Proportion of balls where noballs chance is 0: {len(test_noballs[test_noballs == 0]) / len(test_noballs):.3f}\n")

Mean actual wides chance: 0.692
Mean predicted wides chance: 0.691
Proportion of balls where wides chance is 0: 0.001

Mean actual byes chance: 0.046
Mean predicted byes chance: 0.050
Proportion of balls where byes chance is 0: 0.159

Mean actual legbyes chance: 0.191
Mean predicted legbyes chance: 0.198
Proportion of balls where legbyes chance is 0: 0.065

Mean actual noballs chance: 0.074
Mean predicted noballs chance: 0.078
Proportion of balls where noballs chance is 0: 0.068



Good to see that the chances fairly match up again. Due to the very low number of zeroes, I do not think that adding a fudge factor will be necessary here.

In [47]:
# Wides:
# For the sake of time I am going to limit the max depth of the trees to 30, otherwise I will be sitting here for hours...
X_non_powerplay_wides = non_powerplay_extra_balls.drop(columns=['wides', 'byes', 'legbyes', 'noballs', 'penalty'])
y_non_powerplay_wides = non_powerplay_extra_balls['wides']
X_train, X_test, y_train, y_test = train_test_split(X_non_powerplay_wides, y_non_powerplay_wides, test_size=0.2, random_state=42)
print(f"Non powerplay wides model:")
non_powerplay_wides_model = do_random_forest_classifier(X_train, y_train, max_depth=30)
test_wides = predict_wicket_probability(non_powerplay_wides_model, X_test, 0)

# Byes:
X_non_powerplay_byes = non_powerplay_extra_balls.drop(columns=['wides', 'byes', 'legbyes', 'noballs', 'penalty'])
y_non_powerplay_byes = non_powerplay_extra_balls['byes']
X_train, X_test, y_train, y_test = train_test_split(X_non_powerplay_byes, y_non_powerplay_byes, test_size=0.2, random_state=42)
print(f"Non powerplay byes model:")
non_powerplay_byes_model = do_random_forest_classifier(X_train, y_train, max_depth=30)
test_byes = predict_wicket_probability(non_powerplay_byes_model, X_test, 0)

# Legbyes:
X_non_powerplay_legbyes = non_powerplay_extra_balls.drop(columns=['wides', 'byes', 'legbyes', 'noballs', 'penalty'])
y_non_powerplay_legbyes = non_powerplay_extra_balls['legbyes']
X_train, X_test, y_train, y_test = train_test_split(X_non_powerplay_legbyes, y_non_powerplay_legbyes, test_size=0.2, random_state=42)
print(f"Non powerplay legbyes model:")
non_powerplay_legbyes_model = do_random_forest_classifier(X_train, y_train, max_depth=30)
test_legbyes = predict_wicket_probability(non_powerplay_legbyes_model, X_test, 0)

# Noballs:
X_non_powerplay_noballs = non_powerplay_extra_balls.drop(columns=['wides', 'byes', 'legbyes', 'noballs', 'penalty'])
y_non_powerplay_noballs = non_powerplay_extra_balls['noballs']
X_train, X_test, y_train, y_test = train_test_split(X_non_powerplay_noballs, y_non_powerplay_noballs, test_size=0.2, random_state=42)
print(f"Non powerplay noballs model:")
non_powerplay_noballs_model = do_random_forest_classifier(X_train, y_train, max_depth=30)
test_noballs = predict_wicket_probability(non_powerplay_noballs_model, X_test, 0)

# Penalty:
# See above cell.

Non powerplay wides model:
Model Performance:
Train AUC: 0.989 ± 0.000
CV AUC: 0.608 ± 0.005
Precision: 0.672 ± 0.004
Recall: 0.787 ± 0.007
F1 Score: 0.725 ± 0.004
Non powerplay byes model:
Model Performance:
Train AUC: 0.997 ± 0.000
CV AUC: 0.603 ± 0.007
Precision: 0.185 ± 0.060
Recall: 0.039 ± 0.012
F1 Score: 0.064 ± 0.019
Non powerplay legbyes model:
Model Performance:
Train AUC: 0.995 ± 0.000
CV AUC: 0.655 ± 0.006
Precision: 0.363 ± 0.014
Recall: 0.142 ± 0.008
F1 Score: 0.204 ± 0.010
Non powerplay noballs model:
Model Performance:
Train AUC: 0.995 ± 0.000
CV AUC: 0.604 ± 0.014
Precision: 0.268 ± 0.049
Recall: 0.086 ± 0.016
F1 Score: 0.131 ± 0.023


In [48]:
dump(non_powerplay_wides_model, '../models/simulation_models/non_powerplay_wides_model.joblib')
dump(non_powerplay_byes_model, '../models/simulation_models/non_powerplay_byes_model.joblib')
dump(non_powerplay_legbyes_model, '../models/simulation_models/non_powerplay_legbyes_model.joblib')
dump(non_powerplay_noballs_model, '../models/simulation_models/non_powerplay_noballs_model.joblib')


['../models/simulation_models/non_powerplay_noballs_model.joblib']

In [49]:
print(f"Mean actual wides chance: {non_powerplay_extra_balls['wides'].mean():.3f}")
print(f"Mean predicted wides chance: {test_wides.mean():.3f}")
print(f"Proportion of balls where wides chance is 0: {len(test_wides[test_wides == 0]) / len(test_wides):.3f}\n")

print(f"Mean actual byes chance: {non_powerplay_extra_balls['byes'].mean():.3f}")
print(f"Mean predicted byes chance: {test_byes.mean():.3f}")
print(f"Proportion of balls where byes chance is 0: {len(test_byes[test_byes == 0]) / len(test_byes):.3f}\n")
print(f"Mean actual legbyes chance: {non_powerplay_extra_balls['legbyes'].mean():.3f}")
print(f"Mean predicted legbyes chance: {test_legbyes.mean():.3f}")
print(f"Proportion of balls where legbyes chance is 0: {len(test_legbyes[test_legbyes == 0]) / len(test_legbyes):.3f}\n")

print(f"Mean actual noballs chance: {non_powerplay_extra_balls['noballs'].mean():.3f}")
print(f"Mean predicted noballs chance: {test_noballs.mean():.3f}")
print(f"Proportion of balls where noballs chance is 0: {len(test_noballs[test_noballs == 0]) / len(test_noballs):.3f}\n")

Mean actual wides chance: 0.632
Mean predicted wides chance: 0.629
Proportion of balls where wides chance is 0: 0.000

Mean actual byes chance: 0.080
Mean predicted byes chance: 0.085
Proportion of balls where byes chance is 0: 0.071

Mean actual legbyes chance: 0.201
Mean predicted legbyes chance: 0.207
Proportion of balls where legbyes chance is 0: 0.059

Mean actual noballs chance: 0.090
Mean predicted noballs chance: 0.097
Proportion of balls where noballs chance is 0: 0.026



Again, this seems to match up remarkably well. Also again, due to the very low number of zeroes, I do not think that adding a fudge factor will be needed here.

# 2.c) Predicting the number of extras occuring in a ball | The type of extra

Again, I will be limiting my comments/explanations due to the length of this task.

In [50]:
# We need a regression model rather than a classifier here.
def do_random_forest_regressor(X_train, y_train, cv=5, random_state=42, **kwargs):
    model = RandomForestRegressor(random_state=random_state, **kwargs)
    
    cv_results = cross_validate(
        model,
        X_train,
        y_train,
        cv=cv,
        scoring={
            'r2': 'r2',
            'rmse': 'neg_root_mean_squared_error',
            'mape': 'neg_mean_absolute_percentage_error'
        },
        return_train_score=True
    )

    model.fit(X_train, y_train)
    
    analysis = {
        'r2': {
            'train_avg': cv_results['train_r2'].mean(),
            'train_std': cv_results['train_r2'].std(),
            'cv_avg': cv_results['test_r2'].mean(),
            'cv_std': cv_results['test_r2'].std(),
            'gap': cv_results['train_r2'].mean() - cv_results['test_r2'].mean(),
            'consistency': 'Stable' if cv_results['test_r2'].std() < 0.05 else 'Unstable'
        },
        'rmse': {
            'train_avg': -cv_results['train_rmse'].mean(),
            'train_std': cv_results['train_rmse'].std(),
            'cv_avg': -cv_results['test_rmse'].mean(),
            'cv_std': cv_results['test_rmse'].std(),
            'gap': -cv_results['test_rmse'].mean() - (-cv_results['train_rmse'].mean())
        },
        'mape': {
            'train_avg': -cv_results['train_mape'].mean(),
            'train_std': cv_results['train_mape'].std(),
            'cv_avg': -cv_results['test_mape'].mean(),
            'cv_std': cv_results['test_mape'].std(),
            'gap': -cv_results['test_mape'].mean() - (-cv_results['train_mape'].mean())
        }
    }
    
    print("Cross-Validation Analysis:")
    print("\nR² Scores:")
    print(f"Training Average: {analysis['r2']['train_avg']:.3f} ± {analysis['r2']['train_std']:.3f}")
    print(f"CV Average: {analysis['r2']['cv_avg']:.3f} ± {analysis['r2']['cv_std']:.3f}")
    print(f"Gap (Train-CV): {analysis['r2']['gap']:.3f}")
    print(f"CV Consistency: {analysis['r2']['consistency']}")
    
    print("\nRMSE Scores:")
    print(f"Training Average: {analysis['rmse']['train_avg']:.3f} ± {analysis['rmse']['train_std']:.3f}")
    print(f"CV Average: {analysis['rmse']['cv_avg']:.3f} ± {analysis['rmse']['cv_std']:.3f}")
    print(f"Gap (CV-Train): {analysis['rmse']['gap']:.3f}")
    
    print("\nMAPE Scores:")
    print(f"Training Average: {analysis['mape']['train_avg']:.3f} ± {analysis['mape']['train_std']:.3f}")
    print(f"CV Average: {analysis['mape']['cv_avg']:.3f} ± {analysis['mape']['cv_std']:.3f}")
    print(f"Gap (CV-Train): {analysis['mape']['gap']:.3f}")
    
    feature_importance = pd.DataFrame({
        'Feature': X_train.columns,
        'Importance': model.feature_importances_
    }).sort_values('Importance', ascending=False)
    
    print("\nTop 10 Most Important Features:")
    print(feature_importance.head(10))
    
    return model, analysis, feature_importance

In [51]:
balls_with_extras.head()

,batter,bowler,extras_details,extras,powerplay,extras_type,wides,byes,legbyes,noballs,penalty
251,85d8110a,7bbeac6d,{'wides': 1},1,True,[wides],True,False,False,False,False
254,ff014c34,7bbeac6d,{'legbyes': 1},1,False,[legbyes],False,False,True,False,False
265,ff014c34,7bbeac6d,{'legbyes': 1},1,True,[legbyes],False,False,True,False,False
291,ff014c34,7f6c2dae,{'wides': 1},1,False,[wides],True,False,False,False,False
331,9c376401,94a1827c,{'wides': 1},1,False,[wides],True,False,False,False,False


In [52]:
exploded_extras = balls_with_extras.explode('extras_type')
exploded_extras['extras_from_relevant_type'] = exploded_extras.apply(lambda row: row['extras_details'].get(row['extras_type'], 0), axis=1)
exploded_extras = exploded_extras.drop(columns=['extras_details', 'wides', 'byes', 'legbyes', 'noballs', 'penalty', 'extras'])
exploded_extras.head()

,batter,bowler,powerplay,extras_type,extras_from_relevant_type
251,85d8110a,7bbeac6d,True,wides,1
254,ff014c34,7bbeac6d,False,legbyes,1
265,ff014c34,7bbeac6d,True,legbyes,1
291,ff014c34,7f6c2dae,False,wides,1
331,9c376401,94a1827c,False,wides,1


In [53]:
powerplay_extra_balls = exploded_extras[exploded_extras['powerplay']].drop(columns=['powerplay'])
powerplay_extra_balls = powerplay_extra_balls.merge(powerplay_extra_batter_stats, on='batter', how='left').drop(columns=['batter'])
powerplay_extra_balls = powerplay_extra_balls.merge(powerplay_extra_bowler_stats, on='bowler', how='left').drop(columns=['bowler'])
non_powerplay_extra_balls = exploded_extras[exploded_extras['powerplay'] == False].drop(columns=['powerplay'])
non_powerplay_extra_balls = non_powerplay_extra_balls.merge(non_powerplay_extra_batter_stats, on='batter', how='left').drop(columns=['batter'])
non_powerplay_extra_balls = non_powerplay_extra_balls.merge(non_powerplay_extra_bowler_stats, on='bowler', how='left').drop(columns=['bowler'])

# Wides:
print(f"\nPowerplay wides model:")
X_powerplay_wides = powerplay_extra_balls[powerplay_extra_balls['extras_type'] == 'wides'].drop(columns=['extras_type', 'extras_from_relevant_type'])
y_powerplay_wides = powerplay_extra_balls[powerplay_extra_balls['extras_type'] == 'wides']['extras_from_relevant_type']
X_train, X_test, y_train, y_test = train_test_split(X_powerplay_wides, y_powerplay_wides, test_size=0.2, random_state=42)
powerplay_wides_model, _, _ = do_random_forest_regressor(X_train, y_train, max_depth=30)

# Byes:
print(f"\nPowerplay byes model:")
X_powerplay_byes = powerplay_extra_balls[powerplay_extra_balls['extras_type'] == 'byes'].drop(columns=['extras_type', 'extras_from_relevant_type'])
y_powerplay_byes = powerplay_extra_balls[powerplay_extra_balls['extras_type'] == 'byes']['extras_from_relevant_type']
X_train, X_test, y_train, y_test = train_test_split(X_powerplay_byes, y_powerplay_byes, test_size=0.2, random_state=42)
powerplay_byes_model, _, _ = do_random_forest_regressor(X_train, y_train, max_depth=30)

# Legbyes:
print(f"\nPowerplay legbyes model:")
X_powerplay_legbyes = powerplay_extra_balls[powerplay_extra_balls['extras_type'] == 'legbyes'].drop(columns=['extras_type', 'extras_from_relevant_type'])
y_powerplay_legbyes = powerplay_extra_balls[powerplay_extra_balls['extras_type'] == 'legbyes']['extras_from_relevant_type']
X_train, X_test, y_train, y_test = train_test_split(X_powerplay_legbyes, y_powerplay_legbyes, test_size=0.2, random_state=42)
powerplay_legbyes_model, _, _ = do_random_forest_regressor(X_train, y_train, max_depth=30)

# Noballs and penalties result in a fixed number of extras, so these do not need modeling.


Powerplay wides model:
Cross-Validation Analysis:

R² Scores:
Training Average: 0.738 ± 0.011
CV Average: -0.134 ± 0.041
Gap (Train-CV): 0.873
CV Consistency: Stable

RMSE Scores:
Training Average: 0.389 ± 0.010
CV Average: 0.809 ± 0.036
Gap (CV-Train): 0.420

MAPE Scores:
Training Average: 0.113 ± 0.003
CV Average: 0.261 ± 0.008
Gap (CV-Train): 0.148

Top 10 Most Important Features:
                                      Feature  Importance
5    powerplay_runs_from_relevant_extras_mean    0.151957
3                    powerplay_is_wicket_mean    0.141328
2                        powerplay_total_mean    0.134675
7  powerplay_taken_from_relevant_wickets_mean    0.129054
0                  powerplay_runs_batter_mean    0.120712
6          powerplay_total_runs_conceded_mean    0.112188
1             powerplay_high_scoring_hit_mean    0.111279
4         powerplay_batter_runs_conceded_mean    0.098807

Powerplay byes model:
Cross-Validation Analysis:

R² Scores:
Training Average: 0.849 ± 0.

The results here show that the model is not very good at predicting the number of extras occuring in a ball.

This is likely due to a combination of factors:
1. The number of extras is almsot always 1, so the model struggles to predict different values.
2. The amount of data is fairly limited, so the model is unable to learn well.

I do not think that we will be able to predict this with any model so trying would be a waste of time, however there are a few other options.

1. Just set the number of extras to 1 for all types - as this is almost always the case it should not lead to significant issues.
2. Use a probability distribution to predict the number of extras occuring depending on the type.

In [54]:
print(f"Proportion of wides leading to 1 extra: {powerplay_extra_balls[(powerplay_extra_balls['extras_type'] == 'wides') & (powerplay_extra_balls['extras_from_relevant_type'] == 1)].shape[0] / powerplay_extra_balls[powerplay_extra_balls['extras_type'] == 'wides'].shape[0]:.3f}")
print(f"Proportion of byes leading to 1 extra: {powerplay_extra_balls[(powerplay_extra_balls['extras_type'] == 'byes') & (powerplay_extra_balls['extras_from_relevant_type'] == 1)].shape[0] / powerplay_extra_balls[powerplay_extra_balls['extras_type'] == 'byes'].shape[0]:.3f}")
print(f"Proportion of legbyes leading to 1 extra: {powerplay_extra_balls[(powerplay_extra_balls['extras_type'] == 'legbyes') & (powerplay_extra_balls['extras_from_relevant_type'] == 1)].shape[0] / powerplay_extra_balls[powerplay_extra_balls['extras_type'] == 'legbyes'].shape[0]:.3f}")

Proportion of wides leading to 1 extra: 0.899
Proportion of byes leading to 1 extra: 0.620
Proportion of legbyes leading to 1 extra: 0.846


While the proportions for wides and legbyes scoring 1 extra are quite high, there is still a non-negligable chance of more extras occuring. Byes in particular see a significant chance of >1 extra being scored.

I think therefore the second option will be better.
Since the number of runs that are scored when such extras occur can easily be down to luck rather than skill, fitting a distribution to the data should not be too bad a representation. I think we should still filter by powerplay/non-powerplay here, due to the fact that the less fielders on the field will likely increase the number of extras.

For simplicity, I will just use an empirical distribution here.

In [55]:
powerplay_wides = powerplay_extra_balls[powerplay_extra_balls['extras_type'] == 'wides']['extras_from_relevant_type']
powerplay_byes = powerplay_extra_balls[powerplay_extra_balls['extras_type'] == 'byes']['extras_from_relevant_type']
powerplay_legbyes = powerplay_extra_balls[powerplay_extra_balls['extras_type'] == 'legbyes']['extras_from_relevant_type']
powerplay_wides_probs = get_empirical_distribution(powerplay_wides)
powerplay_byes_probs = get_empirical_distribution(powerplay_byes)
powerplay_legbyes_probs = get_empirical_distribution(powerplay_legbyes)

non_powerplay_wides = non_powerplay_extra_balls[non_powerplay_extra_balls['extras_type'] == 'wides']['extras_from_relevant_type']
non_powerplay_byes = non_powerplay_extra_balls[non_powerplay_extra_balls['extras_type'] == 'byes']['extras_from_relevant_type']
non_powerplay_legbyes = non_powerplay_extra_balls[non_powerplay_extra_balls['extras_type'] == 'legbyes']['extras_from_relevant_type']
non_powerplay_wides_probs = get_empirical_distribution(non_powerplay_wides)
non_powerplay_byes_probs = get_empirical_distribution(non_powerplay_byes)
non_powerplay_legbyes_probs = get_empirical_distribution(non_powerplay_legbyes)

In [56]:
powerplay_wides_probs.to_csv('../models/simulation_dists/powerplay_wides_probs.csv', header=False)
powerplay_byes_probs.to_csv('../models/simulation_dists/powerplay_byes_probs.csv', header=False)
powerplay_legbyes_probs.to_csv('../models/simulation_dists/powerplay_legbyes_probs.csv', header=False)
non_powerplay_wides_probs.to_csv('../models/simulation_dists/non_powerplay_wides_probs.csv', header=False)
non_powerplay_byes_probs.to_csv('../models/simulation_dists/non_powerplay_byes_probs.csv', header=False)
non_powerplay_legbyes_probs.to_csv('../models/simulation_dists/non_powerplay_legbyes_probs.csv', header=False)

# 3. Prediciting the number of runs scored from a ball

In [57]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 685093 entries, 248 to 857198
Data columns (total 30 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   game_id               685093 non-null  object 
 1   date                  685093 non-null  object 
 2   venue                 685093 non-null  object 
 3   location              653311 non-null  object 
 4   gender                685093 non-null  object 
 5   match_type            685093 non-null  object 
 6   innings               685093 non-null  int64  
 7   batting_team          685093 non-null  object 
 8   bowling_team          685093 non-null  object 
 9   batting_team_players  685093 non-null  object 
 10  bowling_team_players  685093 non-null  object 
 11  over                  685093 non-null  int64  
 12  ball_in_over          685093 non-null  int64  
 13  batter                685093 non-null  object 
 14  bowler                685093 non-null  object 
 15  non

In [58]:
runs_data = data[['batter', 'bowler', 'powerplay', 'freehit', 'runs_batter']]
runs_data.head()

,batter,bowler,powerplay,freehit,runs_batter
248,ff014c34,7bbeac6d,True,False,0
249,ff014c34,7bbeac6d,True,False,0
250,ff014c34,7bbeac6d,True,False,1
251,85d8110a,7bbeac6d,True,False,0
252,85d8110a,7bbeac6d,True,False,0


In [59]:
runs_data['runs_batter'].value_counts()

runs_batter
0    325148
1    227687
4     62231
2     46995
6     19950
3      2985
5        95
7         2
Name: count, dtype: int64

The data here is extremely unbalances, making the job of a predictive model difficult.

I think in this situation using an empirical distribution will be the best option. However, I am going to segment the data by powerplay/non-powerplay and then further my considering the skill of the bowler compared to the batter.

I will compare their average runs conceded/made per ball to determine skill levels, and segment the data into 3 groups based on this:
1. Bowler is better than batter
2. Bowler is worse than batter
3. Bowler is about the same as batter

In [60]:
powerplay_runs_data = runs_data[runs_data['powerplay']].drop(columns=['powerplay'])
powerplay_runs_data = powerplay_runs_data.merge(powerplay_batter_stats, on='batter', how='left').drop(columns=['batter'])
powerplay_runs_data = powerplay_runs_data.merge(powerplay_bowler_stats, on='bowler', how='left').drop(columns=['bowler'])
non_powerplay_runs_data = runs_data[~runs_data['powerplay']].drop(columns=['powerplay'])
non_powerplay_runs_data = non_powerplay_runs_data.merge(non_powerplay_batter_stats, on='batter', how='left').drop(columns=['batter'])
non_powerplay_runs_data = non_powerplay_runs_data.merge(non_powerplay_bowler_stats, on='bowler', how='left').drop(columns=['bowler'])

In [61]:
powerplay_runs_data = powerplay_runs_data[['runs_batter', 'powerplay_runs_batter_mean', 'powerplay_batter_runs_conceded_mean', 'freehit']]
non_powerplay_runs_data = non_powerplay_runs_data[['runs_batter', 'non_powerplay_runs_batter_mean', 'non_powerplay_batter_runs_conceded_mean', 'freehit']]

In [62]:
powerplay_runs_data['batter_better'] = powerplay_runs_data['powerplay_runs_batter_mean'] >= powerplay_runs_data['powerplay_batter_runs_conceded_mean'] * 1.2
powerplay_runs_data['bowler_better'] = powerplay_runs_data['powerplay_batter_runs_conceded_mean'] >= powerplay_runs_data['powerplay_runs_batter_mean'] * 1.2
powerplay_runs_data['both_equal'] = ~(powerplay_runs_data['batter_better'] | powerplay_runs_data['bowler_better'])

non_powerplay_runs_data['batter_better'] = non_powerplay_runs_data['non_powerplay_runs_batter_mean'] >= non_powerplay_runs_data['non_powerplay_batter_runs_conceded_mean'] * 1.2
non_powerplay_runs_data['bowler_better'] = non_powerplay_runs_data['non_powerplay_batter_runs_conceded_mean'] >= non_powerplay_runs_data['non_powerplay_runs_batter_mean'] * 1.2
non_powerplay_runs_data['both_equal'] = ~(non_powerplay_runs_data['batter_better'] | non_powerplay_runs_data['bowler_better'])

In [63]:
freehit_powerplay_runs_data = powerplay_runs_data[powerplay_runs_data['freehit']]
non_freehit_powerplay_runs_data = powerplay_runs_data[~powerplay_runs_data['freehit']]

freehit_non_powerplay_runs_data = non_powerplay_runs_data[non_powerplay_runs_data['freehit']]
non_freehit_non_powerplay_runs_data = non_powerplay_runs_data[~non_powerplay_runs_data['freehit']]

In [64]:
freehit_powerplay_better_batter_runs_dist = get_empirical_distribution(freehit_powerplay_runs_data[freehit_powerplay_runs_data['batter_better']]['runs_batter'])
freehit_powerplay_better_bowler_runs_dist = get_empirical_distribution(freehit_powerplay_runs_data[freehit_powerplay_runs_data['bowler_better']]['runs_batter'])
freehit_powerplay_same_runs_dist = get_empirical_distribution(freehit_powerplay_runs_data[freehit_powerplay_runs_data['both_equal']]['runs_batter'])

non_freehit_powerplay_better_batter_runs_dist = get_empirical_distribution(non_freehit_powerplay_runs_data[non_freehit_powerplay_runs_data['batter_better']]['runs_batter'])
non_freehit_powerplay_better_bowler_runs_dist = get_empirical_distribution(non_freehit_powerplay_runs_data[non_freehit_powerplay_runs_data['bowler_better']]['runs_batter'])
non_freehit_powerplay_same_runs_dist = get_empirical_distribution(non_freehit_powerplay_runs_data[non_freehit_powerplay_runs_data['both_equal']]['runs_batter'])

freehit_non_powerplay_better_batter_runs_dist = get_empirical_distribution(freehit_non_powerplay_runs_data[freehit_non_powerplay_runs_data['batter_better']]['runs_batter'])
freehit_non_powerplay_better_bowler_runs_dist = get_empirical_distribution(freehit_non_powerplay_runs_data[freehit_non_powerplay_runs_data['bowler_better']]['runs_batter'])
freehit_non_powerplay_same_runs_dist = get_empirical_distribution(freehit_non_powerplay_runs_data[freehit_non_powerplay_runs_data['both_equal']]['runs_batter'])

non_freehit_non_powerplay_better_batter_runs_dist = get_empirical_distribution(non_freehit_non_powerplay_runs_data[non_freehit_non_powerplay_runs_data['batter_better']]['runs_batter'])
non_freehit_non_powerplay_better_bowler_runs_dist = get_empirical_distribution(non_freehit_non_powerplay_runs_data[non_freehit_non_powerplay_runs_data['bowler_better']]['runs_batter'])
non_freehit_non_powerplay_same_runs_dist = get_empirical_distribution(non_freehit_non_powerplay_runs_data[non_freehit_non_powerplay_runs_data['both_equal']]['runs_batter'])

In [65]:
freehit_powerplay_better_batter_runs_dist.to_csv('../models/simulation_dists/freehit_powerplay_better_batter_runs_dist.csv', header=False)
freehit_powerplay_better_bowler_runs_dist.to_csv('../models/simulation_dists/freehit_powerplay_better_bowler_runs_dist.csv', header=False)
freehit_powerplay_same_runs_dist.to_csv('../models/simulation_dists/freehit_powerplay_same_runs_dist.csv', header=False)
non_freehit_powerplay_better_batter_runs_dist.to_csv('../models/simulation_dists/non_freehit_powerplay_better_batter_runs_dist.csv', header=False)
non_freehit_powerplay_better_bowler_runs_dist.to_csv('../models/simulation_dists/non_freehit_powerplay_better_bowler_runs_dist.csv', header=False)
non_freehit_powerplay_same_runs_dist.to_csv('../models/simulation_dists/non_freehit_powerplay_same_runs_dist.csv', header=False)
freehit_non_powerplay_better_batter_runs_dist.to_csv('../models/simulation_dists/freehit_non_powerplay_better_batter_runs_dist.csv', header=False)
freehit_non_powerplay_better_bowler_runs_dist.to_csv('../models/simulation_dists/freehit_non_powerplay_better_bowler_runs_dist.csv', header=False)
freehit_non_powerplay_same_runs_dist.to_csv('../models/simulation_dists/freehit_non_powerplay_same_runs_dist.csv', header=False)
non_freehit_non_powerplay_better_batter_runs_dist.to_csv('../models/simulation_dists/non_freehit_non_powerplay_better_batter_runs_dist.csv', header=False)
non_freehit_non_powerplay_better_bowler_runs_dist.to_csv('../models/simulation_dists/non_freehit_non_powerplay_better_bowler_runs_dist.csv', header=False)
non_freehit_non_powerplay_same_runs_dist.to_csv('../models/simulation_dists/non_freehit_non_powerplay_same_runs_dist.csv', header=False)

Perfect, we now have distibutions of batter runs scored for each comparitive skill level skill level, during the powerplay and not, and when freehits are occuring or not.

# Current progress:

## We have:
1. A model to predict the probability of a wicket occuring in a ball
2. A few models/distributions to predict extras:
    a. Whether any extras occur
    b. If so, what extras occur
    c. If extras occur, how many extra runs are awarded
3. A set of distributions to predict the number of runs scored by the batter in a ball presuming no wicket occurs.

## Next steps:
Create a simulation of a cricket game.

## Expectations:
While I am not expecting this to get results that are better than the individual models, I am hoping that they may be somewhat comparable. I suspect that increased innacuracy will come from the increased complexity of the simulation vs using a single model, given that for every ball results from several models will be used in run calculations.

I am going to set targets of:
1. Runs: R^2 > 0.2, MAPE < 60%
2. Wickets: R^2 > 0.2, RMSE < 4
(I have used RMSE for the Wickets due to the 0 values that will cause issues with MAPE.)

With the above targets representing a model that is half as good as my original baseline using a linear regression model for the final results of a game. 
If I can improve upon said baseline then I will be incredibly suprised, but these targets represent a good initial goal for the simulation.


In [13]:
data = pd.read_csv('../data/saved_data/simulation_test_data.csv')
batter_stats = pd.read_csv('../data/saved_data/batter_stats.csv')
bowler_stats = pd.read_csv('../data/saved_data/bowler_stats.csv')


In [14]:
test_game = data[(data['game_id'] == data.iloc[0]['game_id']) & (data['innings'] == 1)]

batting_team_players = eval(test_game.iloc[0]['batting_team_players'])
bowling_team_players = eval(test_game.iloc[0]['bowling_team_players'])

batting_team_df = batter_stats[batter_stats['batter'].isin(batting_team_players)].set_index('batter')
bowling_team_df = bowler_stats[bowler_stats['bowler'].isin(bowling_team_players)].set_index('bowler')

final_runs = test_game.iloc[-1]['current_runs']
final_wickets = test_game.iloc[-1]['current_wickets']
final_runs, final_wickets

(np.int64(258), np.int64(5))

In [15]:
%load_ext autoreload
%autoreload 2

sys.path.append('..')
from simulation.cricket_simulation import CricketSimulator

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
def evaluate_simulation(data, batters_df, bowlers_df):
    data_by_games_innings = data.groupby(['game_id', 'innings'])

    game_results = {}
    simulator = CricketSimulator()
    errors = []

    for (game_id, innings), game_data in data_by_games_innings:
        try:
            batting_team_players = eval(game_data.iloc[0]['batting_team_players'])
            bowling_team_players = eval(game_data.iloc[0]['bowling_team_players'])

            batting_team_df = batters_df[batters_df['batter'].isin(batting_team_players)].set_index('batter')
            bowling_team_df = bowlers_df[bowlers_df['bowler'].isin(bowling_team_players)].set_index('bowler')

            simulator.init_teams(batting_team_df, bowling_team_df)
            result = simulator.run_simulation()
            
            game_results[game_id] = {
                'runs_actual': game_data.iloc[-1]['current_runs'],
                'runs_predicted': result['runs'],
                'wickets_actual': game_data.iloc[-1]['current_wickets'],
                'wickets_predicted': result['wickets'],
                'key_events': result['key_events']
            }
        except Exception as e:
            print(f"Error in game {game_id}, innings {innings}:")
            errors.append((game_id, innings, str(e), traceback.format_exc()))
            continue
        
    return game_results, errors

game_results, errors = evaluate_simulation(data, batter_stats, bowler_stats)

KeyError: 'a97c8ec2'
Over: 7, Ball: 1
Error in game 0021ca69-72b5-4080-9e2e-f7cd9e558b47, innings 1:
Error simulating ball: DataFrame indexer is not allowed for .iloc
Consider using .loc for automatic alignment.
Over: 8, Ball: 4
Error in game 0021ca69-72b5-4080-9e2e-f7cd9e558b47, innings 2:
Error simulating ball: DataFrame indexer is not allowed for .iloc
Consider using .loc for automatic alignment.
Over: 9, Ball: 1
Error in game 002795ac-3340-4ccf-9763-865ab0fb8268, innings 1:
Error simulating ball: DataFrame indexer is not allowed for .iloc
Consider using .loc for automatic alignment.
Over: 10, Ball: 6
Error in game 002795ac-3340-4ccf-9763-865ab0fb8268, innings 2:
KeyError: '9ccb6e52'
Over: 12, Ball: 1
Error in game 004eb807-a976-4f8b-a19f-348251347bb8, innings 1:
KeyError: 'fbdc5ab1'
Over: 14, Ball: 6
Error in game 004eb807-a976-4f8b-a19f-348251347bb8, innings 2:
KeyError: '31d220f5'
Over: 15, Ball: 6
Error in game 016c1fa4-1e4b-425a-b0ee-9ff9ffa8592f, innings 1:
Error simulating ba

In [17]:
runs_real, runs_predicted, wickets_real, wickets_predicted = [], [], [], []
for game_id, result in game_results.items():
    runs_real.append(result['runs_actual'])
    runs_predicted.append(result['runs_predicted'])
    wickets_real.append(result['wickets_actual'])
    wickets_predicted.append(result['wickets_predicted'])

print(f"R^2 Runs: {r2_score(runs_real, runs_predicted):.3f}, RMSE Runs: {mean_squared_error(runs_real, runs_predicted, squared=False):.3f}, MAPE Runs: {mean_absolute_percentage_error(runs_real, runs_predicted):.3f}, MAE Runs: {mean_absolute_error(runs_real, runs_predicted):.3f}")
print(f"R^2 Wickets: {r2_score(wickets_real, wickets_predicted):.3f}, RMSE Wickets: {mean_squared_error(wickets_real, wickets_predicted, squared=False):.3f}, MAPE Wickets: {mean_absolute_percentage_error(wickets_real, wickets_predicted):.3f}, MAE Wickets: {mean_absolute_error(wickets_real, wickets_predicted):.3f}")

R^2 Runs: -0.539, RMSE Runs: 51.538, MAPE Runs: 0.438, MAE Runs: 42.264
R^2 Wickets: -0.086, RMSE Wickets: 3.285, MAPE Wickets: 836212132403010.375, MAE Wickets: 2.849


/Users/eden/Library/Caches/pypoetry/virtualenvs/coding-task-BR_zkfTV-py3.12/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/Users/eden/Library/Caches/pypoetry/virtualenvs/coding-task-BR_zkfTV-py3.12/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


## Results

The end results of the simulation are mixed. Overall, it performed better than I expected. Compared to the targets set however, results are more mixed.

The final R^2 Scores for Runs and Wickets are -0.539 and -0.086 respectively. These fall significantly short of the targets set (each >0.2), and their negative values indicate that the models are performing worse than simply predicting the mean of the data. The increased complexity likely leads to the model fitting poorly, even if the RMSE and MAPE average out to better values.

The RMSE and MAPE values do improve over the set targets, being 51.5, 0.438 for runs and 3.285 for the wicket RMSE. While they are not particularly close to the original baseline, they do exceed the target by a non-negligable amount.

Overall, while the simulation in its current form does not perform particularly well, the results are reasonable enough that it is plausible that with further tuning, it could come close to, or possibly even outperform the baseline and other models.
The advantage of a simulation in this form, is that it can be tuned more precisely than a model just predicting the final results of a game, due to the individual parameters that can be adjusted. With more time to refine the simulation and tune parameters and models, I suspect that it's results could improve significantly.